In [1]:
import pandas as pd
import openpyxl as exc
from openpyxl import workbook as wb
from openpyxl import load_workbook
import xlrd
import pyexcel as p
import os
from io import StringIO
import glob

In [2]:
def job_date(df):
    job = str(df.columns[0]).split(' ', 1)[1]
    edit_date = pd.to_datetime(str(df.iloc[0,0]).split(' ')[0], errors='coerce')
#     attendance = str(df.iloc[0,0]).split(' ',1)[1]
    return job, edit_date



In [3]:
completed_path = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\completed proj projections"
c_recents = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\c_recent\compiled_data.xlsx"
on_going_path = ''

In [4]:
files = []
for item in os.listdir(completed_path):
    job_code = item.split(' ')[1]
    files.append(job_code)
files= pd.unique(files)
    
#     full_path = os.path.join(completed_path, item)

In [5]:
latest_file_list = []
for file in files:
    job_files = [os.path.join(completed_path, x) for x in os.listdir(completed_path) if file in x]
    latest_file = max(job_files, key=os.path.getctime)
    latest_file_list.append(latest_file)
latest_file_list = pd.unique(latest_file_list)   

In [6]:
df_list = []
for item in latest_file_list:
    num_sheet = len(pd.ExcelFile(item).sheet_names)
    for sheet in range(0,num_sheet):
        xl = pd.read_excel(item ,sheet_name=sheet)
        df = pd.DataFrame(xl)
        job, date = job_date(df)
        df.columns = df.iloc[2]
        df = df.drop(df.index[:3])
        df = df.iloc[:, :27]
        df['updated'] = date
        df['job'] = job
#         df['attend'] = atend
        df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)
        df = df.dropna(subset=['Qty'])
        df = df.reset_index(drop=True)
#         df = df.loc[:,~df.columns.duplicated()].copy()
#         df = df.drop(df.columns[[26,27,28,29]], axis=1)
#         print(job)
#         print(job)
        #         print(date)
#         df = df.reset_index()
        df_list.append(df)
# full_df = pd.concat(df_list).drop_duplicates().reset_index(drop=True)
        #         print(df.head(3))

In [30]:
full_df = pd.concat(df_list).drop_duplicates().reset_index(drop=True)

In [42]:
full_df = pd.concat(df_list).drop_duplicates().reset_index(drop=True)
full_df['Ph Int'] = full_df['Phase Code'].astype(str).str[0]
full_df = full_df[~full_df['updated'].isnull()]
full_df['year'] = pd.DatetimeIndex(full_df['updated']).year
# full_df['est fix'] = ()
# full_df['Est unit $'] = full_df['Total Budget'] / full_df['Qty']
# full_df['Act unit $'] = full_df['Actual Cost$'] / full_df['HJ Qty Comp']
full_df = full_df.sort_values(by='updated', ascending=False)
# full_df['Phase Code'] = full_df['Phase Code'].astype(str)
# full_df['dup_code'] = df.apply(lambda x:'%s_%s_%s' % (x['Phase Code'],x['Description'],x['job']),axis=1)

In [43]:
full_df = full_df[~full_df['updated'].isnull()]
no_dup = full_df.drop_duplicates(subset=['Phase Code','Description','job'], keep='first')

In [57]:
no_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2619 entries, 3800 to 754
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Phase Code              2619 non-null   object        
 1   Description             2618 non-null   object        
 2   Notes                   0 non-null      object        
 3   Qty                     2619 non-null   float64       
 4   HJ Qty Comp             2619 non-null   float64       
 5   Unit                    2619 non-null   object        
 6   % Comp                  2116 non-null   object        
 7   nan                     0 non-null      float64       
 8   Budget Last Projection  2618 non-null   object        
 9   New Budgets             156 non-null    object        
 10  Total Budget            2619 non-null   object        
 11  Actual Cost$            2619 non-null   object        
 12  Cmtd Cost$              261 non-null    object

In [58]:
no_dup['Qty'] = no_dup['Qty'].astype(float)
no_dup['Actual Cost$'] = no_dup['Actual Cost$'].astype(float)
no_dup['Total Budget'] = no_dup['Total Budget'].astype(float)

C:\Users\willd\AppData\Local\Temp\ipykernel_16408\1252987309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_dup['Qty'] = no_dup['Qty'].astype(float)
C:\Users\willd\AppData\Local\Temp\ipykernel_16408\1252987309.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_dup['Actual Cost$'] = no_dup['Actual Cost$'].astype(float)
C:\Users\willd\AppData\Local\Temp\ipykernel_16408\1252987309.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [59]:
most_recent = no_dup.sort_values(by=['job', 'Phase Code'])
most_recent = most_recent[~most_recent[['Qty','HJ Qty Comp']].eq(0).sum(1).ge(2)]
# df.drop_duplicates(subset=['IDName'], keep='last')


In [60]:
v = most_recent.Description.value_counts()
agg_df = most_recent[most_recent.Description.isin(v.index[v.gt(3)])]

In [61]:
writer = pd.ExcelWriter(c_recents, engine="xlsxwriter")
most_recent.to_excel(writer, sheet_name="mk1")
agg_df.to_excel(writer, sheet_name="agg data")
# money_in.to_excel(writer, sheet_name="income")
# money_out.to_excel(writer, sheet_name="expense")
writer.close()

In [40]:
df_list = []
for item in os.listdir(completed_path):
    full_path = os.path.join(completed_path, item)
    num_sheet = len(pd.ExcelFile(full_path).sheet_names)
    for sheet in range(0,num_sheet):
        
        xl = pd.read_excel(full_path ,sheet_name=sheet)
        df = pd.DataFrame(xl)
        df.columns.values[0] = "1"
#         df['1'] = df['1'].fillna(False)
        pc_list = df.index[df['1'].str.contains('Phase Code', na= False)].tolist()
        if len(pc_list) > 0:
            pc_ind = pc_list[0]
#         if 'Phase Code' in df.iloc[:,0]:   
#             job, date = job_date(df)
#             df.columns = df.iloc[pc_ind]
#             df = df.drop(df.index[:3])
#             df['updated'] = date
#             df['job'] = job
#             # remove non numeric values in this column
#             df = df[df['Phase Code'].apply(lambda x: str(x).isnumeric())]
#             df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)
            df_list.append(df)
full_df = pd.concat(df_list)    



In [62]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 917 entries, 0 to 3796
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Phase Code              917 non-null    object        
 1   Description             917 non-null    object        
 2   Notes                   0 non-null      object        
 3   Qty                     917 non-null    float64       
 4   HJ Qty Comp             917 non-null    float64       
 5   Unit                    917 non-null    object        
 6   % Comp                  913 non-null    object        
 7   nan                     0 non-null      float64       
 8   Budget Last Projection  916 non-null    object        
 9   New Budgets             58 non-null     object        
 10  Total Budget            917 non-null    float64       
 11  Actual Cost$            917 non-null    float64       
 12  Cmtd Cost$              44 non-null     object   